# Sprint 6 · Webinar 20 · Sesión Práctica (Preparación de datos + distribuciones + Git)

En esta sesión práctica aplicarás limpieza y descripción estadística usando el dataset `webinar19_customers_latam.csv` (clientes LatAm, 5.000 filas) utilizado en la sesión teórica.


## Fecha

14 de diciembre de 2025


## Objetivos de la sesión práctica

Al finalizar la sesión, el/la estudiante podrá:

1. Identificar variables relevantes para un análisis estadístico (enfoque churn y comportamiento).
2. Detectar y tratar valores ausentes e inválidos con reglas simples y justificadas.
3. Convertir y estandarizar tipos de datos (numéricos y fechas) usando funciones de Pandas.
4. Usar bucles (`for`, `while`) e iterables para automatizar validaciones repetitivas.
5. Escribir mini-funciones de limpieza reutilizables y bien documentadas.
6. Describir distribuciones numéricas y categóricas (estadísticas + visualizaciones) e interpretar significado de negocio.
7. Crear un repositorio, clonarlo localmente y realizar el primer commit (flujo mínimo Git/GitHub).


## Agenda sugerida (100 minutos)

- Ejercicio 0 · Reto inicial (10 min)
- Ejercicio 1 · Variables relevantes y estructura del dataset (15 min)
- Ejercicio 2 · Valores ausentes e inválidos (20 min)
- Ejercicio 3 · Automatización con funciones de Pandas (15 min)
- Ejercicio 4 · Bucles e iterables para validación (10 min)
- Ejercicio 5 · Mini-funciones propias de limpieza (15 min)
- Ejercicio 6 · Distribuciones: estadística + histogramas + boxplots + negocio (10 min)
- Ejercicio 7 · Git/GitHub: repo + clone + primer commit (5 min)


## Preparación del entorno (ejecuta primero)

Carga librerías y el dataset base. Este dataset tiene columnas "raw" con problemas intencionales para practicar:
- `monthly_fee_raw` con textos como `"N/A"` / `"unknown"`.
- `last_activity_raw` con fechas inválidas o vacías.
- edades fuera de rango (`age`).
- `total_spend_90d_usd` negativo en algunos casos.
- `country` / `city` vacíos o inconsistentes.

Objetivo: convertir esto en un dataset **listo para análisis estadístico**.


In [ ]:
# Librerías base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('display.max_columns', 50)

DATA_PATH = "webinar19_customers_latam.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
df.head()


## Ejercicio 0 · Reto inicial (10 min)

**Contexto:** queremos entender churn (columna `churned`: 1 = churn, 0 = activo).

### Tu reto
Sin hacer todavía una limpieza completa, responde en código:

1. ¿Qué 5 columnas parecen más relevantes para explicar churn? (elige y justifica en 1–2 líneas).
2. Muestra un `df[cols].head()` con esas columnas.
3. Calcula el porcentaje de valores faltantes (`NaN`) por columna y muestra el top 8.

### Pistas
- Usa `df.info()` para ver tipos.
- Usa `df.isna().mean()` para proporciones.
- Piensa en variables de comportamiento: compras, tickets, satisfacción, actividad, plan.


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 0)
# 1) Define una lista con 5 columnas relevantes
# cols_relevantes = [...]

# 2) Muestra una vista rápida
# display(df[cols_relevantes].head())

# 3) Top 8 columnas con más faltantes
# missing = ...
# display(missing.head(8))


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 0)

# 1) Selección razonable de variables para churn (puede variar, aquí un ejemplo)
cols_relevantes = [
    "churned",
    "support_tickets_90d",
    "satisfaction_score",
    "total_spend_90d_usd",
    "orders_90d",
    "plan_type",
    "last_activity_raw"  # útil para crear 'days_since_last_activity'
]

display(df[cols_relevantes].head())

# 2) Valores faltantes (proporción)
missing = df.isna().mean().sort_values(ascending=False)
display(missing.head(8))


## Ejercicio 1 · Identificando variables relevantes (7.1.1) y validación estructural (15 min)

### Objetivo
- Identificar columnas numéricas/categóricas/fechas.
- Encontrar columnas candidatas para análisis de churn y calidad de datos.

### Pistas
- Usa `df.select_dtypes(...)` para separar numéricas y categóricas.
- Revisa cardinalidad con `nunique()`.
- Haz un resumen rápido con `describe()` para numéricas.


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 1)

# 1) Lista columnas numéricas y categóricas
# num_cols = ...
# cat_cols = ...

# 2) Muestra cardinalidad (nunique) para categóricas (top 10, ordenado)
# card = ...
# display(card.head(10))

# 3) describe() para numéricas
# display(df[num_cols].describe().T)


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 1)

num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

print("Numéricas:", num_cols)
print("Categóricas:", cat_cols)

# Cardinalidad para categóricas (útil para entender si hay demasiadas categorías)
card = df[cat_cols].nunique(dropna=False).sort_values(ascending=False)
display(card.head(10))

# Resumen numérico
display(df[num_cols].describe().T)


## Ejercicio 2 · Valores ausentes o inválidos (7.1.2) (20 min)

### Objetivo
Detectar y cuantificar:
- edades inválidas (`age` <= 0 o > 100)
- gasto negativo (`total_spend_90d_usd` < 0)
- `country` / `city` vacíos
- fechas inválidas en `last_activity_raw`

Luego, aplicar una estrategia simple:
- inválidos -> `NaN` (o `NaT` para fechas)  
- imputar edad con mediana  
- reemplazar texto vacío con `"Unknown"`

### Pistas
- `pd.to_datetime(..., errors="coerce")` produce `NaT` si no puede parsear.
- `Series.mask(cond, np.nan)` reemplaza donde la condición es True.
- Para texto: `fillna("").str.strip().replace("", "Unknown")`


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 2)

# 1) Crear máscaras (booleans) de inválidos
# age_invalid = ...
# spend_invalid = ...
# country_empty = ...
# city_empty = ...

# 2) Convertir last_activity_raw a fecha (columna nueva)
# df["last_activity_clean"] = ...

# 3) Crear df_clean con reglas básicas
# df_clean = df.copy()
# - age inválida -> NaN, luego imputar con mediana en age_clean
# - spend negativo -> NaN en spend_clean
# - country/city vacíos -> Unknown
# display(df_clean.head())


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 2)

df_clean = df.copy()

# 1) Máscaras de inválidos
age_invalid = df_clean["age"].isna() | (df_clean["age"] <= 0) | (df_clean["age"] > 100)
spend_invalid = df_clean["total_spend_90d_usd"] < 0

country_empty = df_clean["country"].fillna("").astype(str).str.strip().eq("")
city_empty = df_clean["city"].fillna("").astype(str).str.strip().eq("")

print("Edad inválida o faltante:", age_invalid.mean().round(3))
print("Gasto negativo:", spend_invalid.mean().round(3))
print("Country vacío:", country_empty.mean().round(3))
print("City vacío:", city_empty.mean().round(3))

# 2) Fechas: string -> datetime
df_clean["last_activity_clean"] = pd.to_datetime(df_clean["last_activity_raw"], errors="coerce")

# 3) Limpieza de edad
df_clean["age"] = pd.to_numeric(df_clean["age"], errors="coerce")
df_clean.loc[(df_clean["age"] <= 0) | (df_clean["age"] > 100), "age"] = np.nan
df_clean["age_clean"] = df_clean["age"].fillna(df_clean["age"].median())

# 4) Limpieza de gasto
df_clean["spend_clean"] = df_clean["total_spend_90d_usd"].mask(df_clean["total_spend_90d_usd"] < 0, np.nan)

# 5) País y ciudad
for col in ["country", "city"]:
    df_clean[col] = df_clean[col].fillna("").astype(str).str.strip()
    df_clean[col] = df_clean[col].replace("", "Unknown").str.title()

display(df_clean[["age", "age_clean", "total_spend_90d_usd", "spend_clean", "country", "city", "last_activity_raw", "last_activity_clean"]].head(10))


## Ejercicio 3 · Funciones de Python/Pandas para automatizar limpieza (7.1.3) (15 min)

### Objetivo
- Convertir `monthly_fee_raw` a numérico usando `pd.to_numeric`.
- Crear `monthly_fee_clean` imputando faltantes por **mediana por plan_type**.

### Pistas
- `pd.to_numeric(..., errors="coerce")` convierte textos inválidos a `NaN`.
- `groupby("plan_type")["col"].transform("median")` permite imputar por grupo.


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 3)

# 1) Convertir monthly_fee_raw a numérico en una columna nueva
# df_clean["monthly_fee_num"] = ...

# 2) Imputar por mediana por plan_type
# group_median = ...
# df_clean["monthly_fee_clean"] = ...

# 3) Verifica: ¿cuántos NaN quedan en monthly_fee_clean?
# print(...)
# display(df_clean[["plan_type","monthly_fee_raw","monthly_fee_num","monthly_fee_clean"]].head(10))


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 3)

df_clean["monthly_fee_num"] = pd.to_numeric(df_clean["monthly_fee_raw"], errors="coerce")

group_median = df_clean.groupby("plan_type")["monthly_fee_num"].transform("median")
df_clean["monthly_fee_clean"] = df_clean["monthly_fee_num"].fillna(group_median)

print("NaN en monthly_fee_clean:", df_clean["monthly_fee_clean"].isna().sum())
display(df_clean[["plan_type","monthly_fee_raw","monthly_fee_num","monthly_fee_clean"]].head(12))


## Ejercicio 4 · Bucles (for/while) e iterables (7.1.4) (10 min)

### Objetivo
Crear una tabla de “calidad” por columna (para varias columnas) usando un `for`:

Para cada columna en una lista, calcula:
- porcentaje de faltantes
- mínimo y máximo (si es numérica)
- número de valores únicos (si es categórica)

### Pistas
- Crea una lista `cols_check`.
- Usa `df_clean[col].dtype` o `pd.api.types.is_numeric_dtype(...)`.
- Guarda resultados en una lista de diccionarios y luego `pd.DataFrame(...)`.


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 4)

# cols_check = ["age_clean", "monthly_fee_clean", "orders_90d", "country", "plan_type"]

# results = []
# for col in cols_check:
#     row = {"column": col, "missing_rate": ...}
#     # si numérica -> min/max
#     # si categórica -> nunique
#     results.append(row)

# quality_df = pd.DataFrame(results)
# display(quality_df)


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 4)

from pandas.api.types import is_numeric_dtype

cols_check = ["age_clean", "monthly_fee_clean", "orders_90d", "spend_clean", "country", "plan_type"]

results = []
for col in cols_check:
    row = {"column": col, "missing_rate": float(df_clean[col].isna().mean())}
    if is_numeric_dtype(df_clean[col]):
        row["min"] = float(df_clean[col].min(skipna=True))
        row["max"] = float(df_clean[col].max(skipna=True))
        row["nunique"] = int(df_clean[col].nunique(dropna=True))
    else:
        row["min"] = None
        row["max"] = None
        row["nunique"] = int(df_clean[col].nunique(dropna=False))
    results.append(row)

quality_df = pd.DataFrame(results).sort_values("missing_rate", ascending=False)
display(quality_df)


## Ejercicio 5 · Mini-funciones de limpieza (7.1.5) (15 min)

### Objetivo
Escribe 2 mini-funciones:

1) `compute_days_since_last_activity(df, ref_date)`  
- Convierte `last_activity_clean` (si no existe, créala a partir de `last_activity_raw`).
- Crea columna `days_since_last_activity` respecto a `ref_date` (por defecto `"2025-12-01"`).
- Si la fecha es inválida (`NaT`), deja `NaN`.

2) `flag_high_risk_customers(df)`  
- Crea una columna `risk_flag` (0/1) con una regla simple:
  - `support_tickets_90d >= 3` **y**
  - `satisfaction_score <= 3.5` **y**
  - `days_since_last_activity >= 60`

### Pistas
- `pd.to_datetime(..., errors="coerce")`
- `(ref_date - series).dt.days` para días.
- Documenta la función con un docstring corto.


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 5)

# def compute_days_since_last_activity(df, ref_date="2025-12-01"):
#     """..."""
#     ...

# def flag_high_risk_customers(df):
#     """..."""
#     ...

# df_enriched = compute_days_since_last_activity(df_clean)
# df_enriched = flag_high_risk_customers(df_enriched)

# display(df_enriched[["last_activity_raw","last_activity_clean","days_since_last_activity",
#                      "support_tickets_90d","satisfaction_score","risk_flag"]].head(12))


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 5)

def compute_days_since_last_activity(df_in: pd.DataFrame, ref_date: str = "2025-12-01") -> pd.DataFrame:
    """Crea 'days_since_last_activity' a partir de una fecha de referencia.

    Parámetros
    ----------
    df_in : pd.DataFrame
        DataFrame de entrada.
    ref_date : str
        Fecha de referencia en formato YYYY-MM-DD.

    Retorna
    -------
    pd.DataFrame
        Copia del DataFrame con una columna nueva:
        - days_since_last_activity (float, porque puede tener NaN)
    """
    out = df_in.copy()
    if "last_activity_clean" not in out.columns:
        out["last_activity_clean"] = pd.to_datetime(out["last_activity_raw"], errors="coerce")

    ref = pd.to_datetime(ref_date)
    out["days_since_last_activity"] = (ref - out["last_activity_clean"]).dt.days
    return out


def flag_high_risk_customers(df_in: pd.DataFrame) -> pd.DataFrame:
    """Crea un flag simple de riesgo (0/1) basado en reglas de negocio.

    Regla (ejemplo para práctica):
    - tickets >= 3 y satisfacción <= 3.5 y días sin actividad >= 60.
    """
    out = df_in.copy()

    mask = (
        (out["support_tickets_90d"] >= 3) &
        (out["satisfaction_score"] <= 3.5) &
        (out["days_since_last_activity"] >= 60)
    )
    out["risk_flag"] = mask.astype(int)
    return out


df_enriched = compute_days_since_last_activity(df_clean)
df_enriched = flag_high_risk_customers(df_enriched)

display(df_enriched[["last_activity_raw","last_activity_clean","days_since_last_activity",
                     "support_tickets_90d","satisfaction_score","risk_flag"]].head(12))

print("Proporción de clientes con risk_flag=1:", df_enriched["risk_flag"].mean().round(3))


## Ejercicio 6 · Distribuciones: estadística + visualización + negocio (7.2) (10 min)

### Objetivo
1) Calcular estadísticas numéricas para:
- `spend_clean`, `orders_90d`, `support_tickets_90d`, `satisfaction_score`, `monthly_fee_clean`

2) Calcular frecuencias para:
- `plan_type`, `segment`, `country`

3) Graficar:
- Histograma de `spend_clean`
- Boxplot de `spend_clean` por `plan_type`

4) Interpretación rápida (negocio):
- ¿Qué sugiere la forma de la distribución del gasto?
- ¿Qué plan parece tener mayor variabilidad?

### Pistas
- Numéricas: `describe(percentiles=[...]).T`
- Categóricas: `value_counts(normalize=True)`
- Histogramas: `plt.hist(...)`
- Boxplot por grupos: arma una lista de series y pásala a `plt.boxplot(...)`


In [ ]:
# TU CÓDIGO AQUÍ (Ejercicio 6)

# 1) Estadísticas numéricas
# metrics = [...]
# display(df_enriched[metrics].describe(percentiles=[0.25,0.5,0.75,0.9,0.95]).T)

# 2) Frecuencias categóricas
# for col in [...]:
#     display(...)

# 3) Histogramas / boxplots
# ...


#### Solución:


In [ ]:
# SOLUCIÓN (Ejercicio 6)

metrics = ["spend_clean", "orders_90d", "support_tickets_90d", "satisfaction_score", "monthly_fee_clean"]
display(df_enriched[metrics].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]).T)

for col in ["plan_type", "segment", "country"]:
    freq = df_enriched[col].value_counts(dropna=False)
    prop = (freq / len(df_enriched)).round(3)
    display(pd.DataFrame({"count": freq, "proportion": prop}))

# Histograma de spend
plt.figure(figsize=(8,4))
plt.hist(df_enriched["spend_clean"].dropna(), bins=40)
plt.title("Histograma: spend_clean (USD en 90 días)")
plt.xlabel("USD")
plt.ylabel("Frecuencia")
plt.show()

# Boxplot spend por plan
plt.figure(figsize=(9,4))
groups = []
labels = []
for p in df_enriched["plan_type"].unique():
    groups.append(df_enriched.loc[df_enriched["plan_type"] == p, "spend_clean"].dropna())
    labels.append(p)

plt.boxplot(groups, labels=labels, showfliers=True)
plt.title("Boxplot: spend_clean por plan_type")
plt.xlabel("plan_type")
plt.ylabel("USD (90 días)")
plt.xticks(rotation=20)
plt.show()

# Interpretación rápida (ejemplo)
print("Interpretación (ejemplo): Si el histograma es sesgado a la derecha, pocos clientes concentran gran gasto.")
print("Interpretación (ejemplo): Un boxplot con caja/bigotes más largos indica mayor variabilidad en el gasto.")


## Ejercicio 7 · Git/GitHub: crear repo, clonar, primer commit (8) (5 min)

### Objetivo
Realizar el flujo mínimo de versionamiento para este webinar.

### Checklist (paso a paso)

1. Crear repositorio en GitHub  
   - Nombre sugerido: `sprint5-webinar20-practico`
   - Añade un `README.md` (opcional, recomendado)

2. Clonar en tu computador:
```bash
git clone <URL_DEL_REPO>
cd sprint5-webinar20-practico
```

3. Copiar dentro del repo:
- `Sprint5_Webinar20_Practico.ipynb`
- `webinar19_customers_latam.csv` (o una versión ligera si pesa mucho)

4. Primer commit:
```bash
git add .
git commit -m "Webinar 20: práctica de limpieza y distribuciones"
git push
```

### Pistas comunes
- Si te pide credenciales en HTTPS, usa el método recomendado por GitHub (token personal) o configura SSH (lo veremos con detalle después).
- Verifica instalación:
```bash
git --version
```


## Take aways (resumen extendido)

1. La selección de variables parte de una **pregunta de negocio** (aquí: churn) y se valida con estructura y calidad.
2. Valores faltantes e inválidos no se “tapan” sin pensar: se detectan, cuantifican y se decide una estrategia (eliminar, imputar, marcar).
3. Convertir tipos correctamente (`to_numeric`, `to_datetime`) evita errores silenciosos y acelera el análisis.
4. Los bucles ayudan a automatizar controles repetitivos; para datos grandes, complementa con operaciones vectorizadas.
5. Mini-funciones documentadas son el primer paso hacia pipelines de datos reproducibles.
6. Distribuciones y gráficos convierten números en decisiones: segmentación, foco de soporte, prevención de churn.
7. Git/GitHub te permite trabajar con historial, colaboración y mejores prácticas profesionales desde el inicio.


## Cierre
**Kahoot de repaso (5 min)**
- Practicamos el flujo básico de Git: init, add, commit, push.
- Analizamos distribuciones de datos usando Python.

**Reflexión:**
- ¿Qué ventaja tiene convertir tu trabajo en un proyecto usando funciones y Git en lugar de código suelto?
- ¿Qué te reveló el histograma sobre tus datos hoy?

**Q&A y próximos pasos.**


## Siguientes Pasos
- **Próxima sesión:** Sprint 7 - Segmentación y Outliers profundos.
- **Participación:** Sube tu primer repositorio de práctica a GitHub.
- **Recordatorios:** No temas a la terminal; es tu amiga.
